# Requirements

# Imports

In [1]:
import internetarchive
import os
import tarfile
import csv
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd
import configparser
import mysql.connector
import json

In [2]:
import warnings

# Suppressing the warnings
warnings.filterwarnings('ignore') 

# Functions

## General

In [3]:
def check_folder(name_folder):            
    #Creating folder if that doesn't exist
    p = %pwd
    p = p + f'/{name_folder}'
    path = os.path.expanduser(p)
    if not os.path.exists(path):
        os.makedirs(path)
        print("{} created.".format(path))
        
    return path

In [4]:
def get_file_name(item_name, ext='*tar'):
    
    file_names = [f.name for f in internetarchive.get_files(item_name, glob_pattern= ext)]
    
    return file_names

In [5]:
def humanize(size_bytes):
    KB = 1 << 10
    MB = 1 << 20
    GB = 1 << 30

    if size_bytes < KB:
        return '{} B'.format(size_bytes)
    elif size_bytes < MB:
        return '{:.2f} KiB'.format(size_bytes/KB)
    elif size_bytes < GB:
        return '{:.2f} MiB'.format(size_bytes/MB)
    else:
        return '{:.2f} GiB'.format(size_bytes/GB)


## Cleaning and Transfomations

In [6]:
def get_data(file_path, file_name, word_bag = False, lang = False, db_update = False):
    
    #reading json
    df = pd.read_json(file_path, lines=True, compression='gzip')
    
    #updating total tweets
    if not update_db == False:
        update_db(table = 'control', field = f"count_total = '{len(df)}'", condition = f"name = '{file_name}'")
    
    #Filtering language if a language was sent
    if not lang == False:
        df = df[df.lang == lang]
    
    #taking away columns unnecessary
    df = df.loc[:, ['created_at', 'text', 'entities']]
    
    #filtering Tweets using subject word bag if a word bag was sent
    if not word_bag == False:
        bow = '|'.join(word_bag) # bag of word
        df = df[df['text'].str.contains(bow, case=False)]
        
        #updating filtered tweets
        if not update_db == False:
            update_db(table = 'control', field = f"count_filtered = '{len(df)}'", condition = f"name = '{file_name}'")
    
    return df

## Control Operations

### CSV

In [7]:
def control_op_csv(control_file):

    #if file doesn't exist I'll create it
    if not os.path.exists(control_file):
        with open(control_file, 'w', newline='') as control_csv:
            writer = csv.DictWriter(control_csv, fieldnames=['name','datetime'])
            writer.writeheader()
        control_csv.close()
        print("{} created.".format(control_file))
            
    #Reading the control file
    with open(control_file, 'r', newline='') as control_csv:
        reader = csv.DictReader(control_csv)
        reader_data = [r for r in reader]
        control_csv.seek(0)
    control_csv.close()
    
    return reader_data

In [8]:
def download_file_csv(item_name, file_name, download_folder = 'Downloads'):
    
    path = check_folder(download_folder)
    
    #calling function for getting control file
    control_path = path + '/control_download.csv'    
    reader_data = control_op_csv(control_path)
    
    #Getting Itens names
    item = internetarchive.get_item(item_name)
        
    #openning control file to add file names on the list
    with open(control_path, 'a', newline='') as control_download:
        writer = csv.DictWriter(control_download, fieldnames=['name','datetime'])
        
        #check if the file has been downloaded before
        if not any(row['name'] == file_name for row in reader_data):
            
            #downloading
            r = item.download(
                destdir=path,  # The directory to download files to
                ignore_existing=True,  # Skip files that already exist locally
                checksum=True,  # Skip files based on checksum
                verbose=True,  # Print progress to stdout
                retries=100,  # The number of times to retry on failed requests
                no_directory=True,  # Download withtout the identifier
                files = file_name)
            
            #Adding file name on control list
            row = {'name' : file_name, 'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            writer.writerow(row)
            
    control_download.close()
    
    path_file = path + f'/{file_name}'
    
    return path_file

In [9]:
def tar_file_csv(tar_file, file_name, extration_folder='Extraction'):

    path = check_folder(extration_folder)
    
    #calling function for getting control file
    control_path = path + '/control_extraction.csv'    
    reader_data = control_op(control_path)

    #openning control file to add file names on the list
    with open(control_path, 'a', newline='') as control_tar:
        writer = csv.DictWriter(control_tar, fieldnames=['name','datetime'])
        
        #check if the file has been extrated before
        if not any(row['name'] == file_name for row in reader_data):

            #Extracting file
            tar_file.extract(file_name, path=path)

            #Saving files name on control file
            row = {'name' : file_name, 'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            writer.writerow(row)
            
            path_file = path + f'/{file_name}'
            
        else:
            
            path_file = False
    
    control_tar.close()
    
    
    return path_file

### Mysql

#### Basic operations

In [10]:
def open_connection():

    #getting configuration
    config = configparser.ConfigParser()
    config.read('config.ini')

    # Connect to Mysql
    conn = mysql.connector.connect(
        host = config['mysql']['host'],
        user = config['mysql']['user'],
        password = config['mysql']['password'],
        database = config['mysql']['database']
    )
        
    return conn 

In [11]:
def update_db(table, field, condition):
    
    conn = open_connection()
    cursor = conn.cursor()

    cursor.execute(f"UPDATE {table} SET {field} WHERE {condition};")

    conn.commit()
    cursor.close()
    conn.close()

In [12]:
def query_db(query):
    
    conn = open_connection()
    cursor = conn.cursor()
    
    cursor.execute(query)

    conn.commit()
    cursor.close()
    conn.close()

In [13]:
def read_db(query):
    conn = open_connection()
    
    df = pd.read_sql_query(query, conn)
    
    conn.close()
    return df

#### Specific operations

In [14]:
def check_control_db(file_type, file_name = False):
    
    if file_name == False:
        
        check = read_db(f"SELECT name FROM control WHERE type ='{file_type}'").to_dict(orient='records')
    
    else:
        
        check = read_db(f"SELECT name FROM control WHERE type ='{file_type}' AND name = '{file_name}'").to_dict(orient='records')
        
        if len(check) > 0:
            check = True
        else:
            check = False

    return check

In [15]:
def download_file_db(item_name, file_name, download_folder = 'Downloads', type_control = 'download'):
    
    path = check_folder(download_folder)
    
    #query to get information from Mysql about control
    check = check_control_db(file_type = type_control, file_name = file_name)
                             
    #check if the file has been unzipped before
    if check == False:
    
        #getting Itens names
        item = internetarchive.get_item(item_name)

        #downloading
        r = item.download(
            destdir=path,  # The directory to download files to
            ignore_existing=True,  # Skip files that already exist locally
            checksum=True,  # Skip files based on checksum
            verbose=False,  # Print progress to stdout
            retries=100,  # The number of times to retry on failed requests
            no_directory=True,  # Download withtout the identifier
            files = file_name)
        
        #getting Metadata
        metadata = list(filter(lambda p: p["name"] == file_name, item.item_metadata['files']))

        #Adding file name on control list
        dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
        sz = metadata[0]['size']
        query_db(f"INSERT INTO control (name, datetime, type, size) VALUES ('{file_name}', '{dt}', '{type_control}', '{sz}')")
    
        path_file = path + f'/{file_name}'
        
    else:
        
        path_file = False   
    
    return path_file

In [16]:
def tar_file_db(tar_file, file_name, extration_folder='Extraction', type_control = 'extraction'):
    
    path = check_folder(extration_folder)
    
    #query to get information from Mysql about control
    check = check_control_db(file_type = type_control, file_name = file_name)
    
    
    #check if the file has been unzipped before
    if check == False:

        #Extracting file
        tar_file.extract(file_name, path=path)

        #Adding file name on control list
        dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
        sz = tar_file.getmember(name=file_name).size
        query_db(f"INSERT INTO control (name, datetime, type, size) VALUES ('{file_name}', '{dt}', '{type_control}', '{sz}')")
        
        path_file = path + f'/{file_name}'
    
    else: #case the file has been unzipped
            
        path_file = False
        
    
    return path_file

In [17]:
read_db("SELECT * FROM control WHERE type = 'download'AND name LIKE 'twitter-stream-202208%'")

,idnew_table,name,datetime,type,size,count_total,count_filtered
0,16509,twitter-stream-20220801.tar,2023-05-01 00:06:50,download,2742671360,None,None
1,23434,twitter-stream-20220802.tar,2023-05-01 01:34:20,download,2483517440,None,None
2,30440,twitter-stream-20220803.tar,2023-05-01 02:51:00,download,2592880640,None,None
3,33554,twitter-stream-20220804.tar,2023-05-01 04:05:47,download,2329610240,None,None
4,39144,twitter-stream-20220805.tar,2023-05-01 05:23:17,download,2664581120,None,None
5,41935,twitter-stream-20220806.tar,2023-05-01 06:39:27,download,2590279680,None,None
6,45686,twitter-stream-20220807.tar,2023-05-01 07:55:21,download,2614466560,None,None
7,50671,twitter-stream-20220808.tar,2023-05-01 09:22:42,download,2580101120,None,None
8,54948,twitter-stream-20220809.tar,2023-05-01 12:14:34,download,2678353920,None,None
9,62485,twitter-stream-20220810.tar,2023-05-01 13:44:05,download,2665984000,None,None


In [18]:
read_db("SELECT * FROM control WHERE type = 'extraction' AND name LIKE '20221022%'").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   idnew_table     1440 non-null   int64         
 1   name            1440 non-null   object        
 2   datetime        1440 non-null   datetime64[ns]
 3   type            1440 non-null   object        
 4   size            1440 non-null   object        
 5   count_total     1440 non-null   object        
 6   count_filtered  1440 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 78.9+ KB


In [19]:
len(check_control_db(file_type = 'download'))#, file_name = '20221001/20221001235900.json.gz')

238

# Code

In [29]:
df = read_db("SELECT * FROM control WHERE type = 'extraction'")
down_df = check_control_db(file_type = 'download')

In [30]:
df['data'] = df['name'].str[:8].apply(lambda x: datetime.strptime(x, '%Y%m%d').strftime('%Y-%m-%d'))
df['data'] = pd.to_datetime(df['data'])
df['size'] = df['size'].astype(float)
df['count_total'] = df['count_total'].astype(float)
df['count_filtered'] = df['count_filtered'].astype(float)

In [31]:
donwloaded = df.groupby(pd.Grouper(key='data', freq='M'))['size'].sum()
print(f"Total of days: {len(down_df)} \n")

print(f"Total Downloaded: {humanize(donwloaded.sum())}")
print(f"An average of {humanize(donwloaded.sum()/len(down_df))} per day downloaded \n")

print("Total of Tweets: {:,}".format(df['count_total'].sum()))
print(f"An average of {df['count_total'].sum()/len(down_df):,.0f} tweets downloaded per day \n")

print("Tweets filtered: {:,}".format(df['count_filtered'].sum()))
print(f"An average of {df['count_filtered'].sum()/len(down_df):,.0f} tweets filtered per day \n")

Total of days: 255 

Total Downloaded: 638.98 GiB
An average of 2.51 GiB per day downloaded 

Total of Tweets: 981,453,688.0
An average of 3,848,838 tweets downloaded per day 

Tweets filtered: 336,600.0
An average of 1,320 tweets filtered per day 

